In [ ]:
import numpy as np
from numba import cuda

In [96]:
def summarize_device() -> None:

    device = cuda.get_current_device()

    # Most GPUs have 2048 per SM (NVIDIA spec)
    max_threads_per_sm = 2048
    multiprocessors = device.MULTIPROCESSOR_COUNT
    total_max_threads = max_threads_per_sm * multiprocessors
    max_threads_per_block = device.MAX_THREADS_PER_BLOCK
    max_blocks_per_sm = max_threads_per_sm // max_threads_per_block  
    total_max_blocks = multiprocessors * max_blocks_per_sm  

    print(f"Device: {device.name.decode()}")
    print(f"Multiprocessors (SMs): {multiprocessors}")
    print(f"Max Threads per SM: {max_threads_per_sm}")
    print(f"Total Max Threads: {total_max_threads}")
    print(f"Max Threads per Block: {max_threads_per_block}")
    print(f"Max Blocks per SM: {max_blocks_per_sm}")
    print(f"Total Max Blocks: {total_max_blocks}")

    return


summarize_device()

Device: NVIDIA GeForce RTX 2060 with Max-Q Design
Multiprocessors (SMs): 30
Max Threads per SM: 2048
Total Max Threads: 61440
Max Threads per Block: 1024
Max Blocks per SM: 2
Total Max Blocks: 60


In [ ]:
@cuda.jit
def monte_carlo_simulation(simulation, random_values):
    """
    Monte Carlo simulation using CUDA.
    
    Parameters:
        simulation (device array): Output array to store results and be modified in-place.
        random_values (device array): Pre-generated random values.
    """

    # Get 2D thread indices
    i, j = cuda.grid(2)
    
    # Check bounds to avoid out-of-bounds errors
    if i < simulation.shape[0] and j < simulation.shape[1]:
        simulation[i, j] = random_values[i, j]



In [112]:
# Matrix size
rows, cols = 1_000_000, 100

# Host-side random number generation (only 0 or 1)
random_values = np.random.choice([1, 2], size=(rows, cols)).astype(np.int32)

# Allocate device memory for simulation and random values
d_simulation = cuda.to_device(np.empty((rows, cols), dtype=np.int32))  
d_random_values = cuda.to_device(random_values)

# Define CUDA grid size (Each block has 16x16 threads)
max_threads_per_block: int = cuda.get_current_device().MAX_THREADS_PER_BLOCK
threads_per_block = (int(np.sqrt(max_threads_per_block)), int(np.sqrt(max_threads_per_block)))
blocks_per_grid_x = np.math.ceil(rows / threads_per_block[0])
blocks_per_grid_y = np.math.ceil(cols / threads_per_block[1])
blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

# Launch CUDA kernel
monte_carlo_simulation[blocks_per_grid, threads_per_block](d_simulation, d_random_values)

# Copy results back to host
simulation_result = d_simulation.copy_to_host()

# Print part of the result for verification
print(simulation_result[:5, :5])  # Print a small section


C:\Users\marcu\AppData\Local\Temp\ipykernel_38928\3289763641.py:14: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  blocks_per_grid_x = np.math.ceil(rows / threads_per_block[0])
C:\Users\marcu\AppData\Local\Temp\ipykernel_38928\3289763641.py:15: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  blocks_per_grid_y = np.math.ceil(cols / threads_per_block[1])


[[1 1 2 2 2]
 [1 1 2 1 1]
 [2 2 2 2 2]
 [2 1 1 1 2]
 [2 2 1 2 1]]


In [ ]:
import numpy as np
from numba import cuda, float32
import math
import numba.cuda.random as curand

@cuda.jit
def monte_carlo_option_pricing(S0, K, r, sigma, T, num_paths, rng_states, results):
    """
    CUDA Kernel for Monte Carlo European Option Pricing using Black-Scholes.
    
    Each thread simulates one price path.
    """
    
    # Thread index
    idx = cuda.grid(1)
    
    if idx < num_paths:
        dt = T  # Only one step for European option at expiry
        dW = cuda.random.xoroshiro128p_normal_float32(rng_states, idx) * math.sqrt(dt)
        
        # Simulate asset price at maturity
        S_T = S0 * math.exp((r - 0.5 * sigma ** 2) * dt + sigma * dW)

        # Compute payoff: max(S_T - K, 0)
        results[idx] = max(S_T - K, 0)


In [32]:
# Parameters
S0 = 100                    # Initial stock price
K = 110                     # Strike price
r = 0.05                    # Risk-free rate
sigma = 0.2                 # Volatility
T = 1                       # Time to maturity in years
num_paths = 10_000_000      # Number of Monte Carlo simulations

# Allocate device memory
d_results = cuda.device_array(num_paths, dtype=np.float32)
rng_states = curand.create_xoroshiro128p_states(num_paths, seed=69)

# Configure CUDA kernel
threads_per_block = 32
blocks_per_grid = (num_paths + threads_per_block - 1) // threads_per_block

# Run kernel
monte_carlo_option_pricing[blocks_per_grid, threads_per_block](S0, K, r, sigma, T, num_paths, rng_states, d_results)

# Copy results back to host
results = d_results.copy_to_host()

# Compute option price (discounted expected payoff)
option_price = np.exp(-r * T) * np.mean(results)

print(f"European Call Option Price: {option_price:.4f}")

European Call Option Price: 6.0391
